# Becoming a Data Engineer

## Step 1: Handling Large Data Sets In Python

Download the data sets: https://data.world/moma/exhibitions

In [7]:
import pandas as pd
import numpy as np
# Read the file moma.csv into a dataframe named moma.
moma = pd.read_csv("moma-exhibitions/MoMAExhibitions1929to1989.csv")

In [2]:
# The DataFrame.info() method returns an estimate for the amount of memory a dataframe consumes.
# Display the memory usage of the moma dataframe.
print(moma.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34558 entries, 0 to 34557
Data columns (total 27 columns):
ExhibitionID              34129 non-null float64
ExhibitionNumber          34558 non-null object
ExhibitionTitle           34558 non-null object
ExhibitionCitationDate    34557 non-null object
ExhibitionBeginDate       34558 non-null object
ExhibitionEndDate         33354 non-null object
ExhibitionSortOrder       34558 non-null float64
ExhibitionURL             34125 non-null object
ExhibitionRole            34424 non-null object
ConstituentID             34044 non-null float64
ConstituentType           34424 non-null object
DisplayName               34424 non-null object
AlphaSort                 34424 non-null object
FirstName                 31499 non-null object
MiddleName                3804 non-null object
LastName                  31998 non-null object
Suffix                    157 non-null object
Institution               2458 non-null object
Nationality               26

In [3]:
# Retrieve the underlying BlockManager instance.
print(moma._data)

BlockManager
Items: Index(['ExhibitionID', 'ExhibitionNumber', 'ExhibitionTitle',
       'ExhibitionCitationDate', 'ExhibitionBeginDate', 'ExhibitionEndDate',
       'ExhibitionSortOrder', 'ExhibitionURL', 'ExhibitionRole',
       'ConstituentID', 'ConstituentType', 'DisplayName', 'AlphaSort',
       'FirstName', 'MiddleName', 'LastName', 'Suffix', 'Institution',
       'Nationality', 'ConstituentBeginDate', 'ConstituentEndDate',
       'ArtistBio', 'Gender', 'VIAFID', 'WikidataID', 'ULANID',
       'ConstituentURL'],
      dtype='object')
Axis 1: RangeIndex(start=0, stop=34558, step=1)
FloatBlock: [0, 6, 9, 19, 20, 23, 25], 7 x 34558, dtype: float64
ObjectBlock: [1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 24, 26], 20 x 34558, dtype: object


In [4]:
# Recreate the estimate of the memory footprint by multiplying the number of values
# in the moma dataframe by 8. Assign this number to total_bytes.
# Use the DataFrame.size attribute to return the number of values in a dataframe.
total_bytes = moma.size * 8

# Convert total_bytes from bytes to megabytes, and assign the result to total_megabytes.
total_megabytes = total_bytes / 1048576

print(total_bytes)
print(total_megabytes)

7464528
7.1187286377


In [5]:
# Select just the object columns from the moma dataframe
# and assign the resulting dataframe to obj_cols.
obj_cols = moma.select_dtypes(include=['object'])

# Use the DataFrame.memory_usage() method and set the deep parameter to True
# to return the memory footprint of each column in obj_cols.
# Assign the resulting series to obj_cols_mem.
obj_cols_mem = obj_cols.memory_usage(deep=True)
print(obj_cols_mem)

# Use the Series.sum() method to sum the values in obj_cols_mem,
# convert the result to megabytes, and assign the result to obj_cols_sum.
obj_cols_sum = obj_cols_mem.sum()/1048576
print(obj_cols_sum)

Index                          80
ExhibitionNumber          2085250
ExhibitionTitle           3333695
ExhibitionCitationDate    3577728
ExhibitionBeginDate       2281851
ExhibitionEndDate         2234872
ExhibitionURL             3494606
ExhibitionRole            2179383
ConstituentType           2313112
DisplayName               2548428
AlphaSort                 2534329
FirstName                 2104909
MiddleName                1218917
LastName                  2162937
Suffix                    1110333
Institution               1221368
Nationality               1949664
ArtistBio                 3183300
Gender                    1858994
WikidataID                1821293
ConstituentURL            2677922
dtype: int64
43.766947746276855


In [8]:
# Find the smallest int subtype that can accommodate the values in the ExhibitionSortOrder column.
# Use the Series.astype() function to set the type, and assign it back to the moma dataframe.

col_max = moma['ExhibitionSortOrder'].max()
col_min = moma['ExhibitionSortOrder'].min()

if col_max <  np.iinfo("int8").max and col_min > np.iinfo("int8").min:
    moma['ExhibitionSortOrder'] = moma['ExhibitionSortOrder'].astype("int8")
elif col_max <  np.iinfo("int16").max and col_min > np.iinfo("int16").min:
    moma['ExhibitionSortOrder'] = moma['ExhibitionSortOrder'].astype("int16")
elif col_max <  np.iinfo("int32").max and col_min > np.iinfo("int32").min:
    moma['ExhibitionSortOrder'] = moma['ExhibitionSortOrder'].astype("int32")
elif col_max <  np.iinfo("int64").max and col_min > np.iinfo("int64").min:
    moma['ExhibitionSortOrder'] = moma['ExhibitionSortOrder'].astype("int64")

print(moma['ExhibitionSortOrder'].dtype)
print(moma['ExhibitionSortOrder'].memory_usage(deep=True))

int16
69196


In [9]:
# Convert the remaining float columns to the most space efficient float subtype.
# Select the float columns again, and display their dtypes using the DataFrame.dtypes attribute.

moma['ExhibitionSortOrder'] = moma['ExhibitionSortOrder'].astype("int16")

float_cols = moma.select_dtypes(include=['float'])

print(float_cols.dtypes)

for col in float_cols.columns:
    moma[col] = pd.to_numeric(moma[col], downcast='float')

print(moma.select_dtypes(include=['float']).dtypes)

ExhibitionID            float64
ConstituentID           float64
ConstituentBeginDate    float64
ConstituentEndDate      float64
VIAFID                  float64
ULANID                  float64
dtype: object
Series([], dtype: object)


In [10]:
# Convert the ExhibitionBeginDate and ExhibitionEndDate columns to the datetime type,
# and assign the results back to the moma dataframe.
# Display the memory usage for both of these columns using the DataFrame.memory_usage() method.

moma["ExhibitionBeginDate"] = pd.to_datetime(moma["ExhibitionBeginDate"])
moma["ExhibitionEndDate"] = pd.to_datetime(moma["ExhibitionEndDate"])

print(moma[["ExhibitionBeginDate", "ExhibitionEndDate"]].memory_usage(deep=True))

Index                      80
ExhibitionBeginDate    276464
ExhibitionEndDate      276464
dtype: int64


In [11]:
# Convert all object columns where less than half of
# the column's values are unique to the category dtype.
# Return the deep memory footprint using the DataFrame.info() method.

obj_cols = moma.select_dtypes(include=['object'])

for col in obj_cols.columns:
    num_unique_values = len(moma[col].unique())
    num_total_values = len(moma[col])
    if num_unique_values / num_total_values < 0.5:
        moma[col] = moma[col].astype('category')
        
print(moma.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34558 entries, 0 to 34557
Data columns (total 27 columns):
ExhibitionID              34129 non-null float32
ExhibitionNumber          34558 non-null category
ExhibitionTitle           34558 non-null category
ExhibitionCitationDate    34557 non-null category
ExhibitionBeginDate       34558 non-null datetime64[ns]
ExhibitionEndDate         33354 non-null datetime64[ns]
ExhibitionSortOrder       34558 non-null int16
ExhibitionURL             34125 non-null category
ExhibitionRole            34424 non-null category
ConstituentID             34044 non-null float32
ConstituentType           34424 non-null category
DisplayName               34424 non-null category
AlphaSort                 34424 non-null category
FirstName                 31499 non-null category
MiddleName                3804 non-null category
LastName                  31998 non-null category
Suffix                    157 non-null category
Institution               2458 non-nu

In [13]:
# Read "moma.csv" into a dataframe named moma:
# Set the ExhibitionBeginDate and ExhibitionEndDate columns to the datetime type.
# Include only these columns:
  # ExhibitionID
  # ExhibitionNumber
  # ExhibitionBeginDate
  # ExhibitionEndDate
  # ExhibitionSortOrder
  # ExhibitionRole
  # ConstituentType
  # DisplayName
  # Institution
  # Nationality
  # Gender
# Display the deep memory footprint in megabytes.

keep_cols = ['ExhibitionID', 'ExhibitionNumber', 'ExhibitionBeginDate',
             'ExhibitionEndDate', 'ExhibitionSortOrder', 'ExhibitionRole',
             'ConstituentType', 'DisplayName', 'Institution', 'Nationality', 'Gender']

moma = pd.read_csv("moma-exhibitions/MoMAExhibitions1929to1989.csv",
                   parse_dates=["ExhibitionBeginDate", "ExhibitionEndDate"],
                   usecols=keep_cols)

print(moma.memory_usage(deep=True).sum() / (1024*1024))

14.555106163024902
